# LLM text generation notebook for Google Colab

This notebook uses [https://github.com/oobabooga/text-generation-webui](https://github.com/oobabooga/text-generation-webui) to run conversational models in chat mode.




## Credits

Based on the [original notebook by 81300](https://colab.research.google.com/github/81300/AI-Notebooks/blob/main/Colab-TextGen-GPU.ipynb).

Forked from [Philio](https://github.com/pcrii/Philo-Colab-Collection/blob/main/4bit_TextGen_Gdrive.ipynb).

Find the base Colab notebook used to create this one [here](https://github.com/eucdee/AI)



# Aetherius Ai Assistant

Aetherius Ai Assitant is an Ai personal assistant/companion that can be ran using the Oobabooga Api.  I have set up this collab notebook so those without a GPU can use it.
The main Aetherius Program Should be ran on your main computer.

To connect to the google colab notebook, edit the Host Url located in Aetherius's Config Menu.  Replace "http://localhost:5000/api" with the given Non-Streaming Public URL from this Google Colab.  Leave in the /api.

*Remember to change the runtime type to use GPU under the "Runtime" tab*

## Github:
https://github.com/libraryofcelsus/Aetherius_AI_Assistant

In [ ]:
#@title Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

#@markdown Press play on the music player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
#@title Install the web UI
#remember gradio is currently held back
save_logs_to_google_drive = False #@param {type:"boolean"}
save_everything_to_google_drive = False #@param {type:"boolean"}
#@markdown remember these models are large and free Gdrive is only 15Ggb <br>
install_gptq = True #@param {type:"boolean"}
from IPython.display import clear_output
if save_logs_to_google_drive:
  import os
  import shutil
  from google.colab import drive
  drive.mount('/content/drive')
  base_folder = '/content/drive/MyDrive'

if save_everything_to_google_drive:
    import os
    import shutil
    from google.colab import drive
    drive.mount('/content/drive')
    base_folder = '/content/drive/MyDrive'
    repo_dir = '/content/drive/MyDrive/text-generation-webui'
    model_dir = '/content/drive/MyDrive/text-generation-webui/models'
    gptq_dir = '/content/drive/MyDrive/text-generation-webui/repositories/GPTQ-for-LLaMa'
    if os.path.exists(repo_dir):
        %cd {repo_dir}
        !git pull
    else:
        %cd /content/drive/MyDrive/
        !git clone https://github.com/oobabooga/text-generation-webui

else:
    model_dir = '/content/text-generation-webui/models'
    repo_dir = '/content/text-generation-webui'
    %cd /content
    !git clone https://github.com/oobabooga/text-generation-webui



if save_logs_to_google_drive:
  if not os.path.exists(f"{base_folder}/oobabooga-data"):
    os.mkdir(f"{base_folder}/oobabooga-data")
  if not os.path.exists(f"{base_folder}/oobabooga-data/logs"):
    os.mkdir(f"{base_folder}/oobabooga-data/logs")
  if not os.path.exists(f"{base_folder}/oobabooga-data/softprompts"):
    os.mkdir(f"{base_folder}/oobabooga-data/softprompts")
  if not os.path.exists(f"{base_folder}/oobabooga-data/characters"):
    shutil.move("text-generation-webui/characters", f"{base_folder}/oobabooga-data/characters")
  else:
    !rm -r "text-generation-webui/characters"

  !rm -r "text-generation-webui/softprompts"
  !ln -s "$base_folder/oobabooga-data/logs" "text-generation-webui/logs"
  !ln -s "$base_folder/oobabooga-data/softprompts" "text-generation-webui/softprompts"
  !ln -s "$base_folder/oobabooga-data/characters" "text-generation-webui/characters"

else:
  !mkdir text-generation-webui/logs

!ln -s text-generation-webui/logs .
!ln -s text-generation-webui/characters .
!ln -s text-generation-webui/models .
%rm -r sample_data
%cd text-generation-webui
!wget https://raw.githubusercontent.com/pcrii/Philo-Colab-Collection/main/settings-colab-template.json -O settings-colab-template.json

# Install requirements
!pip install -r requirements.txt
!pip install flask_cloudflared
!pip install -r extensions/google_translate/requirements.txt
!pip install -r extensions/silero_tts/requirements.txt
print(f"\033[1;32;1m\n --> If you see a warning about \"pydevd_plugins\", just ignore it and move on to Step 3. There is no need to restart the runtime.\n\033[0;37;0m")

if install_gptq:
    if save_everything_to_google_drive:
        if os.path.exists(gptq_dir):
            %cd {gptq_dir}
            !git pull
            !pip install ninja
            !pip install -r requirements.txt
            !python setup_cuda.py install

        else:
            !mkdir /content/drive/MyDrive/text-generation-webui/repositories
            %cd /content/drive/MyDrive/text-generation-webui/repositories
            !git clone https://github.com/qwopqwop200/GPTQ-for-LLaMa
            !ln -s GPTQ-for-LLaMa text-generation-webui/repositories/GPTQ-for-LLaMa
            %cd GPTQ-for-LLaMa
            !pip install ninja
            !pip install -r requirements.txt
            !python setup_cuda.py install
    else:
        %mkdir /content/text-generation-webui/repositories/
        %cd /content/text-generation-webui/repositories/
        !git clone https://github.com/qwopqwop200/GPTQ-for-LLaMa
        !mkdir -p text-generation-webui/repositories
        !ln -s GPTQ-for-LLaMa text-generation-webui/repositories/GPTQ-for-LLaMa
        %cd GPTQ-for-LLaMa
        !pip install ninja
        !pip install -r requirements.txt
        !python setup_cuda.py install
!pip install --upgrade accelerate
clear_output()
print("Finished")
if save_logs_to_google_drive or save_everything_to_google_drive:
    drive_NOT_mounted = False
else:
    drive_NOT_mounted = True

if drive_NOT_mounted:
  import os
print("Available Models")
print(os.listdir(model_dir))

In [ ]:
#@title Download Model (Aetherius uses Llama-2-Chat)
#@markdown you can insert any huggingface model in Organization/model format
model_download = "TheBloke/Llama-2-13B-chat-GPTQ" # @param ["TheBloke/Llama-2-13B-chat-GPTQ", "TheBloke/Llama-2-70B-chat-GPTQ"] {allow-input: true}
#@markdown  Download the model
%cd {repo_dir}
!python download-model.py {model_download}
#this lists directorys from your model folder you can copy the name provided for the model you want for use in the the next cell
!rm {model_dir}/place-your-models-here.txt
clear_output()
if save_logs_to_google_drive or save_everything_to_google_drive:
    drive_NOT_mounted = False
else:
    drive_NOT_mounted = True

if drive_NOT_mounted:
  import os
print("Available Models")
print(os.listdir(model_dir))

In [ ]:
#@title Launch
#@markdown Cloudflared may give some errors, as long as you get a non-streaming url, everything should work fine.  If no url is given, re-run this cell.
import json

model_load = "TheBloke/Llama-2-13B-chat-GPTQ" #@param {type:"string"}
path_model_load = model_load.replace("/", "_")
# Parameters
#auto_devices = False #@param {type:"boolean"}
load_4bit_models = True #@param {type:"boolean"}

groupsize_128 = True #@param {type:"boolean"}
load_in_8bit = False #@param {type:"boolean"}
api = True #@param {type:"boolean"}
public_api = True #@param {type:"boolean"}

Llama2_Seq_Length = True #@param {type:"boolean"}
collab_free_tier = True #@param {type:"boolean"}
collab_V_100 = False #@param {type:"boolean"}
collab_A_100 = False #@param {type:"boolean"}

text_streaming = True #@param {type:"boolean"}
activate_silero_text_to_speech = False #@param {type:"boolean"}
activate_sending_pictures = False #@param {type:"boolean"}
activate_character_bias = False #@param {type:"boolean"}
chat_language = "English" # @param ['Afrikaans', 'Albanian', 'Amharic', 'Arabic', 'Armenian', 'Azerbaijani', 'Basque', 'Belarusian', 'Bengali', 'Bosnian', 'Bulgarian', 'Catalan', 'Cebuano', 'Chinese (Simplified)', 'Chinese (Traditional)', 'Corsican', 'Croatian', 'Czech', 'Danish', 'Dutch', 'English', 'Esperanto', 'Estonian', 'Finnish', 'French', 'Frisian', 'Galician', 'Georgian', 'German', 'Greek', 'Gujarati', 'Haitian Creole', 'Hausa', 'Hawaiian', 'Hebrew', 'Hindi', 'Hmong', 'Hungarian', 'Icelandic', 'Igbo', 'Indonesian', 'Irish', 'Italian', 'Japanese', 'Javanese', 'Kannada', 'Kazakh', 'Khmer', 'Korean', 'Kurdish', 'Kyrgyz', 'Lao', 'Latin', 'Latvian', 'Lithuanian', 'Luxembourgish', 'Macedonian', 'Malagasy', 'Malay', 'Malayalam', 'Maltese', 'Maori', 'Marathi', 'Mongolian', 'Myanmar (Burmese)', 'Nepali', 'Norwegian', 'Nyanja (Chichewa)', 'Pashto', 'Persian', 'Polish', 'Portuguese (Portugal, Brazil)', 'Punjabi', 'Romanian', 'Russian', 'Samoan', 'Scots Gaelic', 'Serbian', 'Sesotho', 'Shona', 'Sindhi', 'Sinhala (Sinhalese)', 'Slovak', 'Slovenian', 'Somali', 'Spanish', 'Sundanese', 'Swahili', 'Swedish', 'Tagalog (Filipino)', 'Tajik', 'Tamil', 'Telugu', 'Thai', 'Turkish', 'Ukrainian', 'Urdu', 'Uzbek', 'Vietnamese', 'Welsh', 'Xhosa', 'Yiddish', 'Yoruba', 'Zulu']


activate_google_translate = (chat_language != "English")

language_codes = {'Afrikaans': 'af', 'Albanian': 'sq', 'Amharic': 'am', 'Arabic': 'ar', 'Armenian': 'hy', 'Azerbaijani': 'az', 'Basque': 'eu', 'Belarusian': 'be', 'Bengali': 'bn', 'Bosnian': 'bs', 'Bulgarian': 'bg', 'Catalan': 'ca', 'Cebuano': 'ceb', 'Chinese (Simplified)': 'zh-CN', 'Chinese (Traditional)': 'zh-TW', 'Corsican': 'co', 'Croatian': 'hr', 'Czech': 'cs', 'Danish': 'da', 'Dutch': 'nl', 'English': 'en', 'Esperanto': 'eo', 'Estonian': 'et', 'Finnish': 'fi', 'French': 'fr', 'Frisian': 'fy', 'Galician': 'gl', 'Georgian': 'ka', 'German': 'de', 'Greek': 'el', 'Gujarati': 'gu', 'Haitian Creole': 'ht', 'Hausa': 'ha', 'Hawaiian': 'haw', 'Hebrew': 'iw', 'Hindi': 'hi', 'Hmong': 'hmn', 'Hungarian': 'hu', 'Icelandic': 'is', 'Igbo': 'ig', 'Indonesian': 'id', 'Irish': 'ga', 'Italian': 'it', 'Japanese': 'ja', 'Javanese': 'jw', 'Kannada': 'kn', 'Kazakh': 'kk', 'Khmer': 'km', 'Korean': 'ko', 'Kurdish': 'ku', 'Kyrgyz': 'ky', 'Lao': 'lo', 'Latin': 'la', 'Latvian': 'lv', 'Lithuanian': 'lt', 'Luxembourgish': 'lb', 'Macedonian': 'mk', 'Malagasy': 'mg', 'Malay': 'ms', 'Malayalam': 'ml', 'Maltese': 'mt', 'Maori': 'mi', 'Marathi': 'mr', 'Mongolian': 'mn', 'Myanmar (Burmese)': 'my', 'Nepali': 'ne', 'Norwegian': 'no', 'Nyanja (Chichewa)': 'ny', 'Pashto': 'ps', 'Persian': 'fa', 'Polish': 'pl', 'Portuguese (Portugal, Brazil)': 'pt', 'Punjabi': 'pa', 'Romanian': 'ro', 'Russian': 'ru', 'Samoan': 'sm', 'Scots Gaelic': 'gd', 'Serbian': 'sr', 'Sesotho': 'st', 'Shona': 'sn', 'Sindhi': 'sd', 'Sinhala (Sinhalese)': 'si', 'Slovak': 'sk', 'Slovenian': 'sl', 'Somali': 'so', 'Spanish': 'es', 'Sundanese': 'su', 'Swahili': 'sw', 'Swedish': 'sv', 'Tagalog (Filipino)': 'tl', 'Tajik': 'tg', 'Tamil': 'ta', 'Telugu': 'te', 'Thai': 'th', 'Turkish': 'tr', 'Ukrainian': 'uk', 'Urdu': 'ur', 'Uzbek': 'uz', 'Vietnamese': 'vi', 'Welsh': 'cy', 'Xhosa': 'xh', 'Yiddish': 'yi', 'Yoruba': 'yo', 'Zulu': 'zu'}


%cd {repo_dir}
# Applying the selected language and setting the prompt size to 2048
# if 8bit mode is selected



params = set()
if api:
  params.add('--api')

if collab_free_tier:
  params.add('--gpu-split 14')

if collab_V_100:
  params.add('--gpu-split 15.5')

if collab_A_100:
  params.add('--gpu-split 39.5')

if Llama2_Seq_Length:
  params.add('--max_seq_len 4096')

if public_api:
  params.add('--public-api')

if load_in_8bit:
  params.add('--load-in-8bit')
#if auto_devices:
#  params.add('--auto-devices')
if load_4bit_models:
  params.add('--wbits 4')

if groupsize_128:
  params.add('--groupsize 128')

active_extensions = []
if activate_sending_pictures:
  active_extensions.append('send_pictures')
if activate_character_bias:
  active_extensions.append('character_bias')
if activate_google_translate:
  active_extensions.append('google_translate')
if activate_silero_text_to_speech:
  active_extensions.append('silero_tts')
active_extensions.append('gallery')

if len(active_extensions) > 0:
  params.add(f'--extensions {" ".join(active_extensions)}')

if not text_streaming or activate_google_translate or activate_silero_text_to_speech:
  params.add('--no-stream')
if activate_character_bias:
  params.add('--verbose')

# Starting the web UI
cmd = f"python server.py --share --model-dir {model_dir} --model {path_model_load} --model_type LLaMa --loader exllama {' '.join(params)}"
print(cmd)
!$cmd